# **Experiment 2: Inference and DB Code Execution Notebook**


This notebook is for the 2nd experiment in my final project, where I test Execution Accuracy (EA) via direct code execution of generated SQL vs. a toy SQLITE database.

In [1]:
%%capture
# using unsloth, which is faster and less memory than hugging face
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install evaluate
!pip install tbparse


In [2]:
from datasets import load_dataset, Dataset, concatenate_datasets, list_metrics,load_metric
import pandas as pd
from unsloth import FastLanguageModel,is_bfloat16_supported
import torch
import tensorboard as tb
from tbparse import SummaryReader
import tbparse
import re
from transformers import AutoTokenizer, TrainingArguments, pipeline, TrainerCallback
from random import randint
import numpy as np
import evaluate
from trl import SFTTrainer
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
import os
import sqlite3
import csv
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from google.colab import files

In [4]:
# connect to gdrive to save and retrieve trained models locally
from google.colab import drive
drive_path="/content/gdrive"
drive.mount(drive_path)

Mounted at /content/gdrive


In [5]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # 4-bit quantization

model_dir=f"{drive_path}/MyDrive/models"
checkpoint_dir=f"{drive_path}/MyDrive/checkpoints"
sqlite_dir=f"{drive_path}/MyDrive/sqlite"


## **Create SQLite toy database**

Create database via SQL statements below and sample data loaded from CSV files. The database is composed of tables related to stock market portfolios, with no additional context given to our Llama model other than create table statements

In [6]:
conn = sqlite3.connect('toy.db')
c = conn.cursor() #

# Create table - CLIENTS
# c.execute('''CREATE TABLE test_table
#              ([id] INTEGER PRIMARY KEY, [name] text, [email] text, [joining_date] date, [salary] integer)''')

c.execute('''DROP TABLE IF EXISTS portfolios''')
conn.commit()

c.execute('''DROP TABLE IF EXISTS portfolio_holdings''')
conn.commit()

c.execute('''DROP TABLE IF EXISTS stock_prices''')
conn.commit()

# Create table - PORTFOLIOS
c.execute('''CREATE TABLE IF NOT EXISTS portfolios (portfolio_id INT, portfolio VARCHAR(50), portfolio_owner VARCHAR(50),age INT)''')
conn.commit()

file = open(f"{sqlite_dir}/portfolios.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO portfolios (portfolio_id,portfolio, portfolio_owner,age) VALUES(?, ?, ?,?)"
c.executemany(insert_records, contents)

# Create table - PORTFOLIO HOLDINGS
c.execute('''CREATE TABLE IF NOT EXISTS portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT)''')
conn.commit()

file = open(f"{sqlite_dir}/portfolio_holdings.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO portfolio_holdings (portfolio_id,date,ticker,shares) VALUES(?, ?,?,?)"
c.executemany(insert_records, contents)

# Create table - STOCK PRICES
c.execute('''CREATE TABLE IF NOT EXISTS stock_prices (date DATE,ticker VARCHAR(10),price NUMERIC)''')
conn.commit()

file = open(f"{sqlite_dir}/prices_melted.csv",encoding='utf-8-sig')
contents = csv.reader(file)
insert_records = "INSERT INTO stock_prices (date,ticker,price) VALUES(?, ?,?)"
c.executemany(insert_records, contents)

conn.commit()
conn.close()



SQL context that will be added to every question for this experiment.

In [7]:
sql_context="""CREATE TABLE IF NOT EXISTS portfolios (portfolio_id INT, portfolio VARCHAR(50),portfolio_owner (VARCHAR(50),age INT));
CREATE TABLE portfolio_holdings (portfolio_id INT, date DATE,ticker VARCHAR(10),shares INT);
CREATE TABLE stock_prices (date DATE,ticker VARCHAR(10),price NUMERIC);"""

Get manually prepared Q&A for code execution

In [8]:
questions_df=pd.read_csv(f"{sqlite_dir}/sql_questions_for_code_execution.csv")

In [9]:
!ls

gdrive	sample_data  toy.db


Test database connection and query portfolios table (should return 5 records):

In [23]:
conn = sqlite3.connect("toy.db")

cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM portfolio_holdings JOIN portfolios ON portfolio_holdings.portfolio_id = portfolios.portfolio_id WHERE portfolios.portfolio_owner = 'portfolioB' AND date = '2024-08-01';")
rows = cur.fetchall()

for row in rows:
  print(row)

conn.close()

(0,)


Use the same system prompt format as Experiment 1:

In [11]:
system_message = """You are a text to SQL query translator. Use SQLite dialect only. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA only.
SCHEMA:
{schema}"""

In [12]:
def create_conversation_qa(examples):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sql_context)},
      {"role": "user", "content": examples["question"]},
      {"role": "assistant", "content": "SELECT * from dont matter"}
    ]
  }

In [13]:
qa_dataset = Dataset.from_pandas(questions_df)
qa_dataset=qa_dataset.map(create_conversation_qa, remove_columns=qa_dataset.features,batched = False,)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Check memory:

## **Inference and SQL Query Execution**

In [14]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
0.0 GB of memory reserved.


Load our saved model (Lora adapter config file):

In [15]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"{model_dir}/lora_model", # model named saved locally during training
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Set up the model for inference:

In [16]:
FastLanguageModel.for_inference(model)

In [17]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

Functions for retrieving SQL to execute, as well as executing SQL and returning result

In [18]:
def get_sql(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.01, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    question=sample['messages'][1]['content']
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()

    return question,predicted_answer

In [19]:
def get_sqlite_answer(sql_string):

  conn = sqlite3.connect("toy.db")
  cur = conn.cursor()
  try:
    cur.execute(sql_string)
    print("hello")
    rows = cur.fetchall()
    answer=rows[0][0]
    conn.close()
    if answer is None:
      return 0
    else:
      return answer
  except sqlite3.OperationalError as e:
      # Handle specific SQL execution errors
      print(f"SQL execution error: {e}")
      return "error"
  except sqlite3.DatabaseError as e:
      # Handle general database errors
      print(f"Database error: {e}")
      return "error"
  except Exception as e:
      # Handle other exceptions
      print(f"An unexpected error occurred: {e}")
      return "error"
  finally:
    # Ensure the connection is closed
    if conn:
      conn.close()
  conn.close()


In [20]:
def convert_to_int_or_return(value):
    try:
        float_value = float(value)
        # Attempt to convert the value to an integer
        return int(float_value)
    except ValueError:
        # If a ValueError occurs, return the original value
        return value

Run the inference, collect results, and display accuracy for both the easy and medium/hard questions separately:

In [21]:
qa_easy_success_rate=[]
qa_med_hard_success_rate=[]
easy_error_count=0
med_hard_error_count=0

for s in tqdm(qa_dataset):
    orig_question,generated_sql=get_sql(s)
    print("original question was: ",orig_question)
    print("generated sql was: ",generated_sql)
    actual_answer=convert_to_int_or_return(questions_df.loc[questions_df['question']==orig_question]['answer'].values[0])
    difficulty=questions_df.loc[questions_df['question']==orig_question]['difficulty'].values[0]
    print("actual answer was: ",actual_answer)
    result=convert_to_int_or_return(get_sqlite_answer(generated_sql))
    print("sqlite answer was: ",result)
    if actual_answer==result:
      if difficulty=='easy':
        qa_easy_success_rate.append(1)
      else:
        qa_med_hard_success_rate.append(1)
    elif result=="error":
      if difficulty=='easy':
        qa_easy_success_rate.append(0)
        easy_error_count+=1
      else:
        qa_med_hard_success_rate.append(0)
        med_hard_error_count+=1
    else:
      if difficulty=='easy':
        qa_easy_success_rate.append(0)
      else:
        qa_med_hard_success_rate.append(0)
    if len(qa_easy_success_rate)>0:
      qa_easy_accuracy = sum(qa_easy_success_rate)/len(qa_easy_success_rate)
      print(f"Easy Q&A Accuracy: {qa_easy_accuracy*100:.2f}%")
      print("Easy SQL Execution Error Rate: ",easy_error_count/len(qa_easy_success_rate))
    if len(qa_med_hard_success_rate)>0:
      qa_med_hard_accuracy = sum(qa_med_hard_success_rate)/len(qa_med_hard_success_rate)
      print(f"Med/Hard Q&A Accuracy: {qa_med_hard_accuracy*100:.2f}%")
      print("Med/Hard SQL Execution Error Rate: ",med_hard_error_count/len(qa_med_hard_success_rate))

qa_easy_accuracy = sum(qa_easy_success_rate)/len(qa_easy_success_rate)
qa_med_hard_accuracy = sum(qa_med_hard_success_rate)/len(qa_med_hard_success_rate)

print(f"Easy Q&A Accuracy: {qa_easy_accuracy*100:.2f}%")
print(f"Easy SQL Execution Error Rate: {(easy_error_count/len(qa_easy_success_rate))*100:.2f}%")

print(f"Med/Hard Q&A Accuracy: {qa_med_hard_accuracy*100:.2f}%")
print(f"Med/Hard SQL Execution Error Rate: {(med_hard_error_count/len(qa_med_hard_success_rate))*100:.2f}%")


  1%|          | 1/100 [00:05<08:54,  5.40s/it]

original question was:  How many different portfolios are there?
generated sql was:  SELECT COUNT(DISTINCT portfolio) FROM portfolios;
actual answer was:  5
hello
sqlite answer was:  5
Easy Q&A Accuracy: 100.00%
Easy SQL Execution Error Rate:  0.0


  2%|▏         | 2/100 [00:06<04:35,  2.82s/it]

original question was:  What is the id of portfolioA?
generated sql was:  SELECT portfolio_id FROM portfolios WHERE portfolio = 'portfolioA';
actual answer was:  1
hello
sqlite answer was:  1
Easy Q&A Accuracy: 100.00%
Easy SQL Execution Error Rate:  0.0


  3%|▎         | 3/100 [00:07<03:08,  1.95s/it]

...

original question was:  What is the average stock price for Jan 2024
generated sql was:  SELECT AVG(stock_prices.price) FROM stock_prices INNER JOIN portfolio_holdings ON stock_prices.ticker = portfolio_holdings.ticker WHERE stock_prices.date = '2024-01-01';
actual answer was:  240
hello
sqlite answer was:  0
Easy Q&A Accuracy: 76.00%
Easy SQL Execution Error Rate:  0.16
Med/Hard Q&A Accuracy: 16.67%
Med/Hard SQL Execution Error Rate:  0.16666666666666666


 99%|█████████▉| 99/100 [03:29<00:01,  1.60s/it]

original question was:  What is the max stock price for the whole year
generated sql was:  SELECT MAX(sp.price) FROM stock_prices sp;
actual answer was:  132
hello
sqlite answer was:  price
Easy Q&A Accuracy: 76.00%
Easy SQL Execution Error Rate:  0.16
Med/Hard Q&A Accuracy: 16.33%
Med/Hard SQL Execution Error Rate:  0.16326530612244897


100%|██████████| 100/100 [03:30<00:00,  2.11s/it]

original question was:  What is the min stock price for the whole year
generated sql was:  SELECT MIN(sp.price) FROM stock_prices sp;
actual answer was:  467
hello
sqlite answer was:  132
Easy Q&A Accuracy: 76.00%
Easy SQL Execution Error Rate:  0.16
Med/Hard Q&A Accuracy: 16.00%
Med/Hard SQL Execution Error Rate:  0.16
Easy Q&A Accuracy: 76.00%
Easy SQL Execution Error Rate: 16.00%
Med/Hard Q&A Accuracy: 16.00%
Med/Hard SQL Execution Error Rate: 16.00%
